In [1]:
import os
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./AE_img'):
    os.mkdir('./AE_img')

In [2]:
def normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor

def value_round(tensor):
    return torch.round(tensor)

def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:value_round(tensor))
])
batch_size = 128

dataset = MNIST('./MNIST_dataset', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [3]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(True),
            nn.Linear(256, 64),
            nn.ReLU(True))
        
        self.decoder = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(True),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [5]:
import torchsummary
model = autoencoder().cuda()
torchsummary.summary(model,(96,784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 96, 256]         200,960
              ReLU-2              [-1, 96, 256]               0
            Linear-3               [-1, 96, 64]          16,448
              ReLU-4               [-1, 96, 64]               0
            Linear-5              [-1, 96, 256]          16,640
              ReLU-6              [-1, 96, 256]               0
            Linear-7              [-1, 96, 784]         201,488
           Sigmoid-8              [-1, 96, 784]               0
Total params: 435,536
Trainable params: 435,536
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.29
Forward/backward pass size (MB): 1.99
Params size (MB): 1.66
Estimated Total Size (MB): 3.94
----------------------------------------------------------------


In [6]:
learning_rate, num_epochs = 1e-3, 50
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay=1e-5)
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        
        output = model(img)
        MSE_loss = nn.MSELoss()(output, img)
        
        optimizer.zero_grad()
        MSE_loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0 or epoch == 49 :
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, MSE_loss.item()))
        
        x_gt = to_img(img.cpu().data)
        x_generated = to_img(output.cpu().data)
        
        save_image(x_gt, './AE_img/ground_truth_{}.png'.format(epoch))
        save_image(x_generated, './AE_img/generated_x{}.png'.format(epoch))

torch.save(model.state_dict(), './auto_encoder.pth')

epoch [1/50], loss:0.0333
epoch [11/50], loss:0.0158
epoch [21/50], loss:0.0125
epoch [31/50], loss:0.0116
epoch [41/50], loss:0.0114
